#Step by step design for the two resonator single xmon design. 


## Sections
### I. Transmon only
1. Prepare the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on single mode. <br>
1. Get qubit freq and anharmonicity. <br>
1. Calculate EPR of substrate.  <br>
1. (Extra: Calculate surface EPR.) <br>

### II. Resonator only
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Calculate EPR of substrate. <br>
    
### III. Transmon & resonator
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

### IV. Transmon & resonator
1. Update the layout in qiskit-metal. <br>
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

### V. Analyze the coupled  transmon 2 resonator system. 
1. Finite Element Eigenmode Analysis
1. Identify the mode you want. The mode can inclusively be from 1 to setup.n_modes.
1. Set variables in the Ansys design. As before, we seek 2 modes.
1. Set up the simulation and specify the variables for the sweep.
1. Plot the E-field on the chip's surface.
1. Specify the junctions in the model; in this case there are 2 junctions.
1. Find the electric and magnetic energy stored in the substrate and the system as a whole.
1. Perform EPR analysis for all modes and variations.

In [1]:
%reload_ext autoreload
%autoreload 2


In [4]:

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr
%metal_heading Good news: Qiskit Metal works fine!

In [5]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL

from qiskit_metal.qlibrary.qubits.transmon_pocket_cl_fl import TransmonPocketCL_FL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.couplers.tunable_coupler_01 import TunableCoupler01

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.couplers.line_tee import LineTee
#from qiskit_metal.qlibrary.terminations.launchpad_wb_simplecoupler import LaunchpadWirebondSimple
#from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
import numpy as np
from collections import OrderedDict

from qiskit_metal.qlibrary.qubits.transmon_cross_ebru_temp import TransmonCross_ebru_temp

from qiskit_metal.qlibrary.qubits.transmon_cross_fl_ebru_temp import TransmonCrossFL_ebru_temp
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl_ebru_temp import TransmonCrossFL_CL_ebru_temp


In [6]:
design = metal.designs.DesignPlanar()

design.chips.main.size['size_x'] = '5mm'
design.chips.main.size['size_y'] = '5mm'
design.chips.main.size['size_z'] = '-725um'
gui = MetalGUI(design)

# Resonator length estimate

In [2]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"


find_resonator_length(7
                      , 10, 6, 2)

'8.694776247529553 mm'

In [8]:
design.overwrite_enabled = True
options =  dict(
    connection_pads=dict(
        readout = dict(loc_W=+1, loc_H=+1),
        storage = dict(loc_W=-1, loc_H=-1, )
    ))
    
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_cross_fl_cl import TransmonCrossFL_CL
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl_fl import TransmonPocketCL_FL


# # Q1 = TransmonCrossFL_CL_ebru_temp(design, 'Q1', options = dict(make_extra_rect=False, pos_x = '0.450mm', pos_y='1.410mm', 
# #                                                      cross_length='40um', cross_width='10um', cross_gap='15um',
# #                                                     # cross_gap='3um',
# #                                                  connection_pads = dict(
# #                                                      readout = dict(connector_location = '90', lead_gap='6um',
# #                                                      claw_length='30um', claw_width='5um', ground_spacing='2um', 
# #                                                                     claw_gap='5um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
# #                                                  fl_options = dict(t_shift='3.5um' , t_offset='-6um', t_inductive_gap='4.5um', 
# #                                                                    t_top='10.5um', t_width='5um', t_gap='3um'),
# #                                                  cl_options= dict(c_offset='-40um', c_ground_gap='38um', c_width='5um', 
# #                                                                   c_gap='3um', translate_x='0um', c_angle='90'),
# #                                                  ))


# Q1 = TransmonPocketCL_FL(design, 'Q1', options = dict( pos_x = '0.55mm', pos_y='1.410mm', orientation='90',
#                                                             connection_pads=dict(
#                                     a = dict(pad_gap='20um', loc_W=1.5, loc_H=-1, pad_width='20um', cpw_extend = '50um'),
                                    
                                    
#                                     #b = dict(loc_W=-1, loc_H=-1, pad_width='125um', cpw_extend = '50um', pad_height='60um'),
#                                     #c = dict(loc_W=+1, loc_H=+1, pad_width='110um', cpw_extend = '50um')
#     ),
#     pad_width='88um', pad_height='88um', pad_gap='140um', inductor_width='10um',
#     pocket_width='240um', pocket_height='500um', 
#     cl_pocket_edge = '270',
#     cl_off_center = '2.5um',
#     cl_gap='3um', cl_width='5um',
                                                      
#                                                  ))


Q1 = TransmonPocketCL_FL(design, 'Q1', options = dict( pos_x = '0.2mm', pos_y='-1.09mm', orientation='90',
                                                            connection_pads=dict(
                                    a = dict(pad_gap='20um', loc_W=1.2, loc_H=-1.5, pad_width='25um', cpw_extend = '50um'),
                                    
                                    #b = dict(loc_W=-1, loc_H=-1, pad_width='125um', cpw_extend = '50um', pad_height='60um'),
                                    #c = dict(loc_W=+1, loc_H=+1, pad_width='110um', cpw_extend = '50um')
    ),
    pad_width='88um', pad_height='88um', pad_gap='140um', inductor_width='10um',
    pocket_width='240um', pocket_height='500um', 
    cl_pocket_edge = '90',
    cl_off_center = '0um',
    cl_gap='3um', cl_width='5um',
    cl_length='0um', 
                                                      #fl_options = dict(t_shift='0um', t_offset='130um'),
                                                      
                                                      
                                                 ))


# Q1 = TransmonPocketCL_FL(design, 'Q1', options = dict( pos_x = '0.55mm', pos_y='1.410mm', orientation='90',
#                                                             connection_pads=dict(
#                                     a = dict(pad_gap='20um', loc_W=-1, loc_H=-1, pad_width='50um', cpw_extend = '50um'),
                                    
#                                     #b = dict(loc_W=-1, loc_H=-1, pad_width='125um', cpw_extend = '50um', pad_height='60um'),
#                                     #c = dict(loc_W=+1, loc_H=+1, pad_width='110um', cpw_extend = '50um')
#     ),
#     pad_width='88um', pad_height='88um', pad_gap='140um', inductor_width='10um',
#     pocket_width='240um', pocket_height='500um', 
#     cl_pocket_edge = '270',
#     cl_off_center = '120um',
#     cl_gap='3um', cl_width='5um',
#     cl_length='0um', 
#                                                       #fl_options = dict(t_shift='0um', t_offset='130um'),
                                                      
                                                      
#                                                  ))

Q2 = TransmonCrossFL_CL_ebru_temp(design, 'Q2', options = dict(make_extra_rect=False, make_fl=False, pos_x = '-0.15mm', pos_y='-1.09mm', orientation=0, 
                                                     cross_length='128um', cross_width='36um', cross_gap='30um',
                                                    
                                                 connection_pads = dict(
                                                     readout = dict(connector_location = '90', lead_gap='6um',
                                                     claw_length='90um', claw_width='10um', ground_spacing='3um', 
                                                                    claw_gap='8um', lead_width='10um', lead_length='60um')),#claw_length ='115um', claw_width='25um', claw_gap='7um',ground_spacing='3um')),
                                                 fl_options = dict(t_shift='-100um' , t_offset='0um', t_inductive_gap='4.5um', 
                                                                   t_top='10.5um', t_width='5um', t_gap='3um'),
                                                 cl_options= dict(c_offset='30um', c_ground_gap='38um', c_width='5um', 
                                                                  c_gap='3um', translate_x='-20um', translate_y='-50um',  c_angle='270'),
                                                 rect_options= dict(rect_width='40um', rect_length='8um')))


gui.rebuild()
gui.autoscale()

12:31AM 24s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
12:31AM 25s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?


# Launchpads and charge and flux lines

In [123]:

import numpy as np
from collections import OrderedDict


#LAUNCH CHARGE FOR Q2

launch_chargeQ2_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                           pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um', 
                           pos_x='-2.954mm', pos_y='-1.8mm', orientation='0', lead_length='30um')
lc2 = LaunchpadWirebond(design, 'Launch_ChargeQ2', options = launch_chargeQ2_options)
#anchors1ch = OrderedDict()
#anchors1ch[0] = np.array([1.78, 0.114])
charge_line_Q2 = RoutePathfinder(design,'Charge_Line_2', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='Launch_ChargeQ2',
                                                    pin='tie')),
                                            lead=Dict(
                                                start_straight='175um',
                                                end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors1ch
                                            ))

#LAUNCH 6 (not used for the moment)
launch6 = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954', pos_y='0.60081mm', orientation='180', lead_length='30um')
l6 = LaunchpadWirebond(design, 'Launch6', options = launch6)




#LAUNCH FEED LEFT (for the transmission line)

launch_feed_left = dict(pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='-2.954mm', pos_y='1.8mm', orientation='0', lead_length='30um')

lfl = LaunchpadWirebond(design, 'Launch_Feed_Left', options = launch_feed_left)



launch_feed_right = dict(pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954mm', pos_y='1.80081mm', orientation='180', lead_length='30um')
lfr = LaunchpadWirebond(design, 'Launch_Feed_Right', options = launch_feed_right)


#LAUNCH 2(not used)

launch2 = dict(trace_width= '10um',pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='-2.954mm', pos_y='-0.6mm', orientation='0', lead_length='30um')
l2 = LaunchpadWirebond(design, 'Launch2', options = launch2)

#LAUNCH 3 (not used)
launch3 = dict(trace_width= '10um', pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='-2.954mm', pos_y='0.6mm', lead_length='30um', orientation='0')
l3 = LaunchpadWirebond(design, 'Launch3', options = launch3)



#LAUNCH FLUX FOR QUBIT 1
launch_fluxQ1_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954mm', pos_y='-1.8mm', orientation='180', lead_length='30um')
lf1 = LaunchpadWirebond(design, 'Launch_FluxQ1', options = launch_fluxQ1_options)
# anchors2f = OrderedDict()
# anchors2f[0] = np.array([1.2, 1.25])
# anchors2f[1]=np.array([1.2, 0.5])
flux_line_Q1 = RoutePathfinder(design,'Flux_Line_1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_FluxQ1',
                                                    pin='tie')),
                                            lead=Dict(
                                                 #start_straight='175um',
                                                 #end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '20um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors=anchors2f,
                                            ))


#LAUNCH CHARGE FOR Q1

launch_chargeQ1_options = dict(trace_width= '5um', trace_gap='3um', pad_width='150um', pad_gap_y= '130um', 
                             pad_gap_x = '96um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.954mm', pos_y='-0.59919mm', orientation='180', lead_length='30um')

lc = LaunchpadWirebond(design, 'Launch_Charge_Q1', options = launch_chargeQ1_options)
#anchors1ch = OrderedDict()
#anchors1ch[0] = np.array([1.78, 0.114])
charge_line_Q1 = RoutePathfinder(design,'Charge_Line_Q1', options = dict(hfss_wire_bonds = True,
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='Charge_Line'),
                                                end_pin=Dict(
                                                    component='Launch_Charge_Q1',
                                                    pin='tie')),
                                            lead=Dict(
                                                start_straight='175um',
                                                end_straight = '100um',
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '60um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors1ch
                                            ))

gui.rebuild()    
gui.autoscale()   





04:35AM 22s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
04:35AM 22s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
04:35AM 22s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


# Fixing Ec's for both qubits (and qubit parameters for Xmon)

# Pocket fluxonium (Q1), getting EC pretending that it is a transmon 


In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb1 = EPRanalysis(design, "hfss")

eig_qb1.sim.setup.vars.Lj = '10 nH'
eig_qb1.sim.setup.vars = Dict({'Lj': '10 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

In [ ]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q1'].options['hfss_inductance'] = 'Lj'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb1.sim.setup.max_passes = 16
eig_qb1.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1)
eig_qb1.sim.setup.min_passes = 10
eig_qb1.sim.setup.n_modes = 1

In [ ]:
eig_qb1.sim.renderer._options['max_mesh_length_jj'] = '3um'
eig_qb1.sim.renderer._options['max_mesh_length_port'] = '3um'

In [ ]:
eig_qb1.sim.run(name="fluxonium_only2", components=['Q1'], open_terminations=[], box_plus_buffer = False)
eig_qb1.sim.plot_convergences()

In [ ]:
eig_qb1.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qb1.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'
eig_qb1.run_epr()

# Xmon (Q2)

In [ ]:
eig_qb2 = EPRanalysis(design, "hfss")

eig_qb2.sim.setup.vars.Lj = '10.9 nH'
eig_qb2.sim.setup.vars = Dict({'Lj': '10.9 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

In [ ]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb2.sim.setup.max_passes = 16


eig_qb2.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1)
eig_qb2.sim.setup.min_passes = 10
eig_qb2.sim.setup.n_modes = 1

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb2.sim.renderer._options['max_mesh_length_jj'] = '3um'
eig_qb2.sim.renderer._options['max_mesh_length_port'] = '3um'

In [ ]:
eig_qb2.sim.run(name="Xmon_only", components=['Q2'], open_terminations=[], box_plus_buffer = False)
eig_qb2.sim.plot_convergences()

In [ ]:
eig_qb2.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_qb2.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'
eig_qb2.run_epr()

# Getting qubit-qubit coupling between the pocket fluxonium and the Xmon 

# With pyepr, again pretending that both qubits are transmons and assuming that Jc is simply capacitive coupling 

# with capacitance matrix

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

# example: update single setting
c1.sim.setup.max_passes = 10
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

c1.sim.run(components=['Q1', 'Q2'], open_terminations=[])
c1.sim.capacitance_matrix

# Flux Lines and Flux Launchpads

In [ ]:
print("RUn")

# Feedline


In [261]:


ops_oF = {'hfss_wire_bonds': 'True',
            'pin_inputs': 
            {'start_pin': {'component': 'Tf_q2', 'pin': 'prime_start'}, 
             'end_pin': {'component': 'Launch_Feed_Left', 'pin': 'tie'}},
            'fillet':'80um'
            }
cpw_openFirst = RouteStraight(design, 'cpw_openFirst', options=ops_oF)




ops_oM = {'hfss_wire_bonds': 'False',
            'pin_inputs': 
            {'start_pin': {'component': 'Tf_q2', 'pin': 'prime_end'}, 
             'end_pin': {'component': 'Launch_Feed_Right', 'pin': 'tie'}},
            'fillet':'50um'
            }
cpw_openMid = RouteStraight(design, 'cpw_openMid', options=ops_oM)



Tf_q2 = CapNInterdigitalTee(design, 'Tf_q2', options=dict(pos_x='-1.1mm',
                                             pos_y='1.8mm',
                                            orientation=0, finger_count='12', finger_length='60um', cap_gap='3um', cap_width='6um', 
                                                 open_termination=False))

gui.rebuild()
gui.autoscale()



12:15AM 26s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:15AM 26s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:15AM 26s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
12:15AM 26s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:15AM 26s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


# Purcell filter


In [276]:
options_purcell2 = Dict(
     total_length='9.07mm',
     hfss_wire_bonds = True,
     fillet='40um',
     lead = dict(start_straight='250um'),
     pin_inputs=Dict(
         end_pin=Dict(
             component='TQ2',
             pin='second_end'),
         start_pin=Dict(
             component='Tf_q2',
             pin='second_end')),
     meander=Dict(spacing='200um',
             asymmetry='0um'))
purcell2 = RouteMeander(design, 'purcell2', options=options_purcell2)

gui.rebuild()
gui.autoscale()

12:51AM 30s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
12:51AM 30s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:51AM 30s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [277]:

eig_prcl = EPRanalysis(design, "hfss")



qcomps=design.components



eig_qb2.sim.setup.max_passes = 3

eig_qb2.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1)
eig_qb2.sim.setup.min_passes = 2
eig_qb2.sim.setup.n_modes = 2

gui.rebuild()
gui.autoscale()


eig_qb2.sim.run(name="purcell_resonator", components=['purcell2', 'TQ2', 'Tf_q2' ], open_terminations=[('TQ2', 'prime_start'), ('TQ2', 'prime_end'), ('Tf_q2', 'prime_start'), ('Tf_q2', 'prime_end')], box_plus_buffer = True)
eig_qb2.sim.plot_convergences()


12:51AM 35s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
12:51AM 35s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:51AM 35s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
INFO 12:51AM [connect_design]: 	Opened active design
	Design:    purcell_resonator_hfss [Solution type: Eigenmode]


KeyboardInterrupt: 

# Readout Tee's and Readout Resonators

In [278]:
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
TQ2 = CapNInterdigitalTee(design, 'TQ2', options=dict(pos_x='-2.9mm',
                                             pos_y='-0.25mm',
                                            orientation=180, finger_length='30um', finger_count='6',cap_gap='3um', cap_width='5um',
                                                 open_termination=False))
#TQT=LineTee(design, 'TQT',options=dict(pos_x='-2.9mm', pos_y='-0.10mm'))

otgQ2 = OpenToGround(design, 'otgQ2', options = dict(pos_x='-4mm', 
                                                   pos_y='-0.35mm', orientation='270', termination_gap='6um', gap='6um'))

#Coupler_resonator2 = CapNInterdigitalTee(design, 'Coupler_resonator2', options=dict(pos_x='-1.1mm', pos_y='-0.09mm'))


options2_part1 = Dict(
     total_length='7.5mm',
     fillet='30um',
    hfss_wire_bonds=False,
     lead = dict(start_straight='500um'),
     pin_inputs=Dict(
         end_pin=Dict(
             component='TQ2',
             pin='prime_start'),
         start_pin=Dict(
             component='Q2',
             pin='readout')),
     meander=Dict(spacing='200um',
             asymmetry='0um'))

options2_part2 = Dict(
     total_length='1.9mm',
     hfss_wire_bonds = False,
     fillet='40um',
     lead = dict(start_straight='300um', end_straight='200um'),
     pin_inputs=Dict(
         end_pin=Dict(
             component='otgQ2',
             pin='open'),
         start_pin=Dict(
             component='TQ2',
             pin='prime_end')),
     meander=Dict(spacing='300um',
             asymmetry='0um'))
read_q2_part1 = RouteMeander(design, 'read_q2_part1', options=options2_part1)
read_q2_part2 = RouteMeander(design, 'read_q2_part2', options=options2_part2)



options2_part2 = Dict(
     total_length='1mm',
     hfss_wire_bonds = False,
     fillet='40um',
     #lead = dict(start_straight='300um', end_straight='200um'),
     pin_inputs=Dict(
         end_pin=Dict(
             component='TQT',
             pin='second_end'),
         start_pin=Dict(
             component='TQ2',
             pin='second_end')),
     meander=Dict(spacing='300um',
             asymmetry='0um'))

gui.rebuild()
gui.autoscale()


12:51AM 48s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
12:51AM 49s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:51AM 49s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [279]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb2 = EPRanalysis(design, "hfss")

eig_qb2.sim.setup.vars.Lj = '10.9 nH'
eig_qb2.sim.setup.vars = Dict({'Lj': '10.9 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'


eig_qb2.sim.setup.max_passes = 20

eig_qb2.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1)
eig_qb2.sim.setup.min_passes = 15
eig_qb2.sim.setup.n_modes = 3

gui.rebuild()
gui.autoscale()


eig_qb2.sim.renderer._options['max_mesh_length_jj'] = '3um'
eig_qb2.sim.renderer._options['max_mesh_length_port'] = '3um'





12:51AM 55s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
12:51AM 56s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:51AM 56s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:51AM 56s INFO [make_connection_pad]: Warning: Did you mean to define a transmon wubit with loc_W and loc_H that are not +1 or -1?? Are you sure you want to do this?
12:51AM 56s WARNING [check_lengths]: For path table, component=cpw_openMid, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
12:51AM 56s WARNING [check_lengths]: For path table, component=cpw_openMid, key=cut 

In [280]:

eig_qb2.sim.run(name="Xmon_and_resonator2", components=['Q2','read_q2_part1','read_q2_part2', 'otgQ2', 'TQ2' , 
                                                        'purcell2','Tf_q2' ], 
                open_terminations=[('Tf_q2', 'prime_end'), ('Tf_q2', 'prime_start')], box_plus_buffer = True)

#eig_qb2.sim.run(name="Xmon_and_resonator", components=['Q2','read_q2_part1','read_q2_part2', 'otgQ2', 'TQ2' , 'TQT'
#                                                         ], 
#                open_terminations=[('TQT', 'prime_end'), ('TQT', 'prime_start')], box_plus_buffer = True)

eig_qb2.sim.plot_convergences()


INFO 12:52AM [connect_design]: 	Opened active design
	Design:    Xmon_and_resonator2_hfss [Solution type: Eigenmode]
INFO 12:52AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 12:52AM [analyze]: Analyzing setup Setup
12:55AM 34s INFO [get_f_convergence]: Saved convergences to C:\Users\WANGLAB\Documents\GitHub\qiskit-metal\tutorials\4 Analysis\A. Core - EM and quantization\hfss_eig_f_convergence.csv


In [260]:

eig_qb2.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_qb2.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'
eig_qb2.run_epr()

Design "Xmon_and_resonator_hfss" info:
	# eigenmodes    3
	# variations    1
Design "Xmon_and_resonator_hfss" info:
	# eigenmodes    3
	# variations    1

        energy_elec_all       = 9.72275996147231e-24
        energy_elec_substrate = 8.95614918564908e-24
        EPR of substrate = 92.1%

        energy_mag    = 9.72197481136592e-24
        energy_mag % of energy_elec_all  = 100.0%
        

Variation 0  [1/1]

  Mode 0 at 5.34 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.1%  9.554e-25 8.562e-27

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.61%
	jj               1.02777  (+)        0.0251897
		(U_tot_cap-U_tot_ind)/mean=-0.56%
Calculating Qdielectric_main for mode 0 (0/2)


  sol = sol.append(self.get_Qdielectric(



p_dielectric_main_0 = 0.9178366936230699

  Mode 1 at 5.97 GHz   [2/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
                0.3%  1.024e-24 1.021e-24

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_1j   sign s_1j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.03%
	jj              0.00319076  (+)        9.78164e-05
		(U_tot_cap-U_tot_ind)/mean=-0.00%
Calculating Qdielectric_main for mode 1 (1/2)


  sol = sol.append(self.get_Qdielectric(



p_dielectric_main_1 = 0.91765769263625

  Mode 2 at 10.93 GHz   [3/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
                0.0%  4.861e-24 4.861e-24

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_2j   sign s_2j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 90.67%
	jj              3.53353e-07  (+)        3.63422e-08
		(U_tot_cap-U_tot_ind)/mean=0.00%
Calculating Qdielectric_main for mode 2 (2/2)


...

###### Numerical diagonalization

Lj,10.9
0,5090.71
1,5967.67
2,10931.84


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1         2
Lj                                  
10.9 0  2.59e+02  7.51e-01  3.43e-04
     1  7.51e-01  7.87e-04  6.43e-07
     2  3.43e-04  6.43e-07  2.40e-10

In [ ]:

TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='1.07mm',
                                             pos_y='2.6mm',
                                              coupling_length='400um', coupling_space='4um', open_termination=False, orientation='0'))

options1 = Dict(
    total_length='3.9mm',
    hfss_wire_bonds = True,
    fillet='40um',
    lead = dict(start_straight='50um'),
    pin_inputs=Dict(
        start_pin=Dict(
            component='TQ1',
            pin='second_end'),
        end_pin=Dict(
            component='Q1',
            pin='a')))
read_q1 = RouteMeander(design, 'read_q1', options=options1)
gui.rebuild()
gui.autoscale()

# Feedline and Feedline Launchpads 

In [ ]:
launch_feed_left_options = dict(trace_width= '10um',pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='-1.5mm', pos_y='3.8mm', orientation='0', lead_length='30um')
l_feed_left = LaunchpadWirebond(design, 'Launch_Feed_Left', options = launch_feed_left_options)


launch_feed_right_options = dict(trace_width= '10um', pad_width='150um', pad_gap_x= '96um', pad_gap_y= '130um', pad_height='200um', 
                           taper_height='150um',  pos_x='1.5mm', pos_y='3.8mm', lead_length='30um', orientation='180')
l_feed_right = LaunchpadWirebond(design, 'Launch_Feed_Right', options = launch_feed_right_options)




# anchors1c = OrderedDict()
# anchors1c[0] = np.array([-0.65, 0.56079])



ops_oF = {'hfss_wire_bonds': 'True',
            'pin_inputs': 
            {'start_pin': {'component': 'TQ2', 'pin': 'prime_start'}, 
             'end_pin': {'component': 'Launch_Feed_Left', 'pin': 'tie'}},
            #'fillet':'80um'
            }
cpw_openFirst = RouteStraight(design, 'cpw_openFirst', options=ops_oF)




ops_oM = {'hfss_wire_bonds': 'True',
            'pin_inputs': 
            {'start_pin': {'component': 'TQ1', 'pin': 'prime_start'}, 
             'end_pin': {'component': 'TQ2', 'pin': 'prime_end'}},
            #'anchors': anchors1c,
            #'fillet':'50um'
            }
cpw_openMid = RouteStraight(design, 'cpw_openMid', options=ops_oM)





# ops_oLast = {'hfss_wire_bonds': 'True',
#             'pin_inputs': 
#             {'start_pin': {'component': 'TQ1', 'pin': 'prime_end'}, 
#              'end_pin': {'component': l_feed_right.name, 'pin': 'tie'}},
#             'fillet':'80um'
#             }
# cpw_openLast = RouteStraight(design, 'cpw_openLast', options=ops_oLast)



gui.rebuild()
gui.autoscale()

In [ ]:
%metal_heading Now analyzing the design: HFSS & EPR

# Running for an individual XMon

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb2 = EPRanalysis(design, "hfss")

eig_qb2.sim.setup.vars.Lj = '10.9 nH'
eig_qb2.sim.setup.vars = Dict({'Lj': '10.9 nH', 'Cj': '3 fF'})
gui.rebuild()
gui.autoscale()

In [ ]:
design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb.sim.setup

In [ ]:
eig_qb.sim.setup.max_passes = 16
eig_qb.sim.setup.vars.Lj = '10 nH'
eig_qb.sim.setup.vars = Dict({'Lj': '10 nH', 'Cj': '3 fF'})

eig_qb.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 4)
eig_qb.sim.setup.min_passes = 10
eig_qb.sim.setup.n_modes = 1

gui.rebuild()
gui.autoscale()



In [ ]:
eig_qb.sim.renderer._options['max_mesh_length_jj'] = '3um'
eig_qb.sim.renderer._options['max_mesh_length_port'] = '3um'

In [ ]:

design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()



eig_qb.sim.run(name="Xmon_only_new", components=['Q2'], open_terminations=[], box_plus_buffer = True)
eig_qb.sim.plot_convergences()

The last variables you pass to the `run()` or `sim.run()` methods, will be stored in the `sim.setup` dictionary under the key `run`. You can recall the information passed by either accessing the dictionary directly, or by using the print handle below.

In [ ]:
eig_qb.sim.plot_convergences()

In [ ]:
# eig_qb.setup.run    <- direct access
eig_qb.sim.print_run_args()

In [ ]:
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'
eig_qb.run_epr()

# Capacitance matrix 

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

# example: update single setting
c1.sim.setup.max_passes = 10
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

c1.sim.run(components=['Q1', 'Q2'], open_terminations=[])
c1.sim.capacitance_matrix

# Qubit 2 readout simulation -EBRU

# Qubit 2 related simulations

In [ ]:
eig_rd.sim.setup.n_modes =1

eig_rd.sim.setup.min_passes = 1
eig_rd.sim.setup.max_passes = 1

# eig_rd.sim.setup.min_passes = 11
# eig_rd.sim.setup.max_passes = 17


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q2'].options['hfss_inductance'] = 'Lj'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()






#eig_rd.sim.run(name="storage_coupling_PART92", components=['Q2','Store_Q2_2', 'Storage_Line_2', 'otg11_2', 'stg11_2'], open_terminations=[('Storage_Line_2', 'end')], box_plus_buffer = False)




#eig_rd.sim.run(name="xmon_and_readout_storage_ayoy_39", components=['Q2', 'read_q2', 'Store_Q2_2', 'TQ2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.run(name="xmon_and_readout_PART90", components=['Q2', 'read_q2', 'TQ2'], open_terminations=[('TQ2', 'prime_start'), ('TQ2', 'prime_end')], box_plus_buffer = False)


#eig_rd.sim.run(name="readout4", components=['read_q1', 'TQ1', 'Q1'], open_terminations=[('TQ1', 'prime_end'), ('TQ1', 'prime_start')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_and_xmon_and_readout", components=['Q1', 'Store_Q1', 'read_q1', 'TQ1', 'Launch_Feed_Left', 'Launch_Feed_Right', 'cpw_openRight', 'cpw_openLeft'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="xmon_chargeline", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1','end')])
# eig_rd.sim.run(name="open_24_131", components=['Q1', 'Store_Q1_1', 'Launch_Storage'], open_terminations=[('Launch_Storage','tie')], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_coupling_ayoy", components=['Q1', 'Store_Q1_1', 'stg11', 'Storage_Line_1', 'otg11'], open_terminations=[('otg11', 'open'), ('Storage_Line_1', 'end')], box_plus_buffer = False)
#eig_rd.sim.run(name="ro_111622", components=['Q2', 'read_q2', 'TQ2', 'Store_Q2_2'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="full_set2", components=['Q2', 'Store_Q2_2' 'read_q2', 'TQ2'], open_terminations=[], box_plus_buffer = False)

#eig_rd.sim.run(name="xmon_and_ro_set1", components=['Q1', 'Store_Q1_1', 'read_q1', 'TQ1'], open_terminations=[], box_plus_buffer = False)
#eig_rd.sim.run(name="storage_loss_fluxpad2", components=['Q2', 'Store_Q2_2', 'Launch_Flux2'], open_terminations=[], box_plus_buffer = False)
eig_rd.sim.plot_convergences()

In [ ]:
eig_rd.sim.setup.n_modes =3

eig_rd.sim.setup.min_passes = 1
eig_rd.sim.setup.max_passes = 1

# eig_rd.sim.setup.min_passes = 15
# eig_rd.sim.setup.max_passes = 17


eig_rd.sim.setup_update(max_delta_f = 0.5, min_freq_ghz = 1.1)


design.qgeometry.tables['junction']
qcomps=design.components
qcomps['Q1'].options['hfss_inductance'] = 'Lj'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj'

gui.rebuild()
gui.autoscale()


eig_rd.sim.run(name="chargecoupling", components=['Q1', 'Charge_Line_1'], open_terminations=[('Charge_Line_1', 'end')], box_plus_buffer = False)


eig_rd.sim.plot_convergences()

In [ ]:
eig_rd.sim.setup

Recover eigenmode frequencies for each variation.

Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:

hfss = eig_rd.sim.renderer
hfss.clear_fields(['Mag_E1'])
hfss.set_mode(1, "Setup")
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')
hfss.save_screenshot()

### EPR Analysis
Find the electric and magnetic energy stored in the readout system.

In [ ]:

eig_rd.sim.setup.name

In [ ]:
# eig_rd.setup
# eig_rd.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
# eig_rd.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

# eig_rd.setup

eig_rd.setup
eig_rd.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_rd.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'

eig_rd.setup

In [ ]:

eig_rd.run_epr()

In [ ]:
eig_rd.sim.setup.vars.Lj 

In [ ]:
eig_rd.sim.close()

# LOM

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

# example: update single setting
c1.sim.setup.max_passes = 10
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup


c1.sim.run(components=['Q1', 'Store_Q1_1','Storage_Line_1', 'Launch_Storage'], open_terminations=[('Storage_Line_1', 'start')])


In [ ]:
c1.sim.capacitance_matrix